# Sparseloop Tutorial


First, include some libraries

In [ ]:
# Run boilerplate code to set up environment

%run ./prelude.py --style=tree --animation=movie

# Matrix Multiply

## Traverse a rank-2 tensor (dense, sparse)

## Configure rank-2 tensors

In [ ]:
K = 8
M = 6
N = 6

tm2 = TensorMaker("sparseloop-matrix-multiply", autoload=True)

tm2.addTensor("A", rank_ids=["M", "K"], shape=[M, K], density=0.4, color="blue")
tm2.addTensor("B", rank_ids=["K", "N"], shape=[K, N], density=0.5, color="green")

tm2.displayControls()


## Create and display rank-2 tensors

In [ ]:
A_MK = tm2.makeTensor("A")
B_KN = tm2.makeTensor("B")

displayTensor(A_MK)
displayTensor(B_KN)

# Traverse a rank-2 tensor

In [ ]:
#
# Create a rank-2 tensor
#
A_MK = tm2.makeTensor("A")

#
# Get root of tensor
#
a_m = A_MK.getRoot()

#
# Animation bookkeeeping
#
canvas = createCanvas(A_MK)

cycle = 0

#
# Traverse non-empty elements of top rank
#
for m, a_k in a_m:
    #
    # Traverse non-empty element of bottom rank
    #
    for k, a_val in a_k:
        
        print(f"{a_val} ", end='')
            
        #
        # Animation bookkeeping
        #
        canvas.addActivity((m,k), spacetime=(0,cycle))
        cycle += 1
            
displayCanvas(canvas)

# Matrix multiply

$$ Z_{m,n} = A_{m,k} \times B_{k,n} $$

In [ ]:
#
# Create tensors
#
A_MK = tm2.makeTensor("A")
B_KN = tm2.makeTensor("B")

M = A_MK.getShape("M")
K = A_MK.getShape("K")
N = B_KN.getShape("N")

Z_MN = Tensor(name="Z", rank_ids=["M", "N"], shape=[M, N])

uncompressTensor(Z_MN)

#
# Display the tensors
#
print(f"M: {M}")
print(f"K: {K}")
print(f"N: {N}")

displayTensor(A_MK)
displayTensor(B_KN)
displayTensor(Z_MN)

#
# Get the root fibers of each tensor
#
a_m = A_MK.getRoot()
b_k = B_KN.getRoot()
z_m = Z_MN.getRoot()

#
# Animation bookkeeping
#
canvas = createCanvas(A_MK, B_KN, Z_MN)

cycle = 0

#
# Traverse non-empty elements of top rank of `A`
# creating the corresponding output fiber in `Z`
#
for m, a_k in a_m:
    #
    # Traverse the K rank of `A` and `B` with matchings coordinates
    #
    for k, a_val in a_k:
        #
        # Obtain the matching fiber in `B`
        #
        b_n = b_k.getPayload(k)
        
        #
        # Traverse the bottom rank of `B`
        # creating an cooresponding output value in the bottom rank of `Z`
        #
        for n, b_val in b_n:
            #
            # Do the reduction
            #
            z_m[m][n] += a_val * b_val
            
            #
            # Animation bookkeeping
            #
            canvas.addActivity((m,k), (k,n), (m, n), spacetime=(0,cycle))
            cycle += 1
            
displayTensor(Z_MN)
displayCanvas(canvas)

# Tiled Matrix Multiply

$$ Z_{n1,m,n0} = A_{m,k} \times B_{n1,k,n0}$$

## Splitting of B

In [ ]:
#
# Create tensors
#

N0 = 2

B_KN = tm2.makeTensor("B")
displayTensor(B_KN)

B_N1N0K = B_KN.splitUniform(N0, depth=1)
B_N1N0K = B_N1N0K.updateCoords(lambda n, c, p: n, depth=1)
displayTensor(B_N1N0K)

B_N1KN0 = B_N1N0K.swapRanks()
displayTensor(B_N1KN0)

## Tiled Matrix Multiply

In [ ]:
#
# Create tensors
#

A_MK = tm2.makeTensor("A")
B_KN = tm2.makeTensor("B")

M = A_MK.getShape("M")
K = A_MK.getShape("K")
N = B_KN.getShape("N")

N1 = 2
N0 = (N+1)//2

B_N1KN0 = B_KN.splitUniform(N0, depth=1).swapRanks()
B_N1KN0 = B_N1KN0.updateCoords(lambda n, c, p: n)

Z_MN = Tensor(name="Z",
              rank_ids=["M", "N"],
                 shape=[M, N])
uncompressTensor(Z_MN)

Z_N1MN0 = Z_MN.splitUniform(N0, depth=1).swapRanks()
Z_N1MN0 = Z_N1MN0.updateCoords(lambda n, c, p: n)
Z_N1MN0.setMutable(True)


#
# Display Tensors
#
print(f"M: {M}")
print(f"K: {K}")
print(f"N1: {N1}")
print(f"N0: {N0}")

displayTensor(A_MK)
displayTensor(B_N1KN0)
displayTensor(Z_N1MN0)

#
# Get the root fibers of each tensor
#
a_m = A_MK.getRoot()
b_n1 = B_N1KN0.getRoot()
z_n1 = Z_N1MN0.getRoot()

#
# Animation bookkeeping
#
canvas = createCanvas(A_MK, B_N1KN0, Z_N1MN0)

cycle = 0

#
# Traverse non-empty elements of top rank of `A`
# creating the corresponding output fiber in `Z`
#
for n1, b_k in b_n1:
    for m, a_k in a_m:
        #
        # Traverse the K rank of `A` and `B` with matchings coordinates
        #
        for k, a_val in a_k:
            #
            # Obtain the matching fiber in `B`
            #
            b_n0 = b_k.getPayload(k)
        
            #
            # Traverse the bottom rank of `B`
            # creating an cooresponding output value in the bottom rank of `Z`
            #
            for n0, b_val in b_n0:
                #
                # Do the reduction
                #
                # Note hack to get right position in N0 rank
                #
                z_n1[n1][m][n0%N0] += a_val * b_val
            
                #
                # Animation bookkeeping
                #
                canvas.addActivity([(m,k)], [(n1,k,n0)], [(n1, m, n0)], 
                                   spacetime=(0,cycle))
                cycle += 1
            
displayTensor(Z_N1MN0)
displayCanvas(canvas)

## Testing area

For running alternative algorithms